## Load libraries

In [1]:
import json
import urllib
import requests
import polling

## Set token/URL

The Python `requests` library expects a single URL, rather than a URL plus a path, hence we define a URL that includes the path to the simulation calls.

In [2]:
token_file = "token.json"
base_path = "https://covid-modelling-stg.epcc.ed.ac.uk/api"
simulations_url = "/".join([base_path, "simulations"])

In [3]:
with open(token_file,'r') as f:
    token = json.load(f)["token"]

In [4]:
headers = {"accept":"application/json; charset=utf-8", "Content-Type": "application/json", "Authorization": "Bearer "+ token}

## Input

In [5]:
inputJSON = {
  "model_slug": "sir-ode-python",
  "config": {
    "p": [
      0.5,
      0.25
    ],
    "u0": [
      0.99,
      0.01,
      0
    ],
    "tspan": [
      0,
      10000
    ]
  }
}

These are parameters to poll for output (every `step` seconds) and to timeout the simulation.

In [12]:
step = 60
timeout = 600

## Post simulations

In [6]:
post_simulations_response = requests.post(simulations_url, json = inputJSON, headers = headers)

In [7]:
post_simulations_response.status_code

200

In [9]:
post_simulations_content = post_simulations_response.json()
post_simulations_content

{'id': 127}

## Check simulations

In [17]:
check_simulations_url = "/".join([simulations_url,str(post_simulations_response.json()["id"])])
check_simulations_url

'https://covid-modelling-stg.epcc.ed.ac.uk/api/simulations/127'

In [21]:
check_simulations_response = requests.get(check_simulations_url, headers = headers)

In [23]:
check_simulations_response.status_code

200

In [24]:
check_simulations_content = check_simulations_response.json()
check_simulations_content

{'id': 127,
 'github_user_id': 2164195,
 'github_user_login': 'sdwfrost',
 'region_name': None,
 'subregion_name': None,
 'region_id': None,
 'subregion_id': None,
 'configuration': {'p': [0.5, 0.25],
  'u0': [0.99, 0.01, 0],
  'tspan': [0, 10000]},
 'model_runs': [{'status': 'unsupported',
   'model_slug': 'basel',
   'results_data': '',
   'export_location': ''},
  {'status': 'unsupported',
   'model_slug': 'mrc-ide-covid-sim',
   'results_data': '',
   'export_location': ''},
  {'status': 'unsupported',
   'model_slug': 'idm-covasim',
   'results_data': '',
   'export_location': ''},
  {'status': 'unsupported',
   'model_slug': 'wss',
   'results_data': '',
   'export_location': ''},
  {'status': 'complete',
   'model_slug': 'sir-ode-python',
   'results_data': 'simulation-runs/Wl2t9N8Ms5hKqQu8KV0glf1OqlViATn+m4SkOu90TZU=/public/data.json',
   'export_location': 'simulation-runs/Wl2t9N8Ms5hKqQu8KV0glf1OqlViATn+m4SkOu90TZU=/public/export.zip'},
  {'status': 'unsupported',
   'model_s

## Export simulation output

In [10]:
export_simulations_params = {
    "model": inputJSON["model_slug"],
    "format": "results"
}

In [11]:
export_simulations_url = "/".join([simulations_url,str(post_simulations_response.json()["id"]),"export"])
export_simulations_url

'https://covid-modelling-stg.epcc.ed.ac.uk/api/simulations/127/export'

The following code polls the API to check for success every `step` seconds, with a `timeout` in seconds.

In [15]:
def is_200_response(response):
    """Check that the response returned 'success'"""
    return response.status_code == 200

try:
    export_simulations_response = polling.poll(
        lambda: requests.get(export_simulations_url, params = export_simulations_params, headers = headers),
        check_success=is_200_response,
        step=step,
        timeout=timeout)
except polling.TimeoutException:
    print(export_simulations_url+" failed")

In [16]:
export_simulations_content = export_simulations_response.json()
export_simulations_content

{'metadata': {'p': [0.5, 0.25], 'u0': [0.99, 0.01, 0], 'tspan': [0, 10000]},
 'model': {'name': 'sir-ode-python',
  'modelVersion': '',
  'connectorVersion': ''},
 't': [0,
  0.09284972135797114,
  1.0213469349376825,
  4.730308099259784,
  10.22867768926573,
  18.314252615167373,
  27.827530514766856,
  37.199522058929524,
  43.77195890487287,
  50.34439575081621,
  57.3485144317926,
  64.42717783070341,
  71.96730511161991,
  80.42082880780592,
  90.4035138669975,
  102.75262015167876],
 'u': [[0.99,
   0.9895352372107644,
   0.9842749739892699,
   0.9474778577501857,
   0.8111285685695783,
   0.4692702181049498,
   0.2718634256070883,
   0.2237105381716237,
   0.213843871904752,
   0.21009771771198776,
   0.20861408640735243,
   0.20807773246099143,
   0.20788008766351446,
   0.20781024841779758,
   0.20778823424147266,
   0.2077826379984259],
  [0.01,
   0.01022997899299957,
   0.012825206057771092,
   0.030572187075163206,
   0.08927463371474981,
   0.157367415842732,
   0.0705566